In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# LOAD DATASET
csv_path = "/content/drive/MyDrive/PBL SEM 5/Dataset ML/V2/DATASET/dataset_feature_engineered.csv"
df = pd.read_csv(csv_path)

print("\n===== Info Dataset =====")
print(df.info())


===== Info Dataset =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   No_Reff                9000 non-null   int64  
 1   Nominal_Transaksi      9000 non-null   int64  
 2   Jenis_Transaksi        9000 non-null   object 
 3   Timestamp              9000 non-null   object 
 4   Nama_Pengirim          9000 non-null   object 
 5   Nama_Penerima          9000 non-null   object 
 6   Tanggal                9000 non-null   object 
 7   Bulan                  9000 non-null   int64  
 8   Hari                   9000 non-null   int64  
 9   Hari_Minggu            9000 non-null   int64  
 10  Minggu_Ke              9000 non-null   int64  
 11  Is_Weekend             9000 non-null   int64  
 12  Is_Akhir_Bulan         9000 non-null   int64  
 13  Is_Awal_Bulan          9000 non-null   int64  
 14  Quarter                9000 no

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

print("="*60)
print("PERSIAPAN DATA EWS (Minimal Features untuk Inferensi)")
print("="*60)

# -----------------------------------------------------------
# 1. DROP KOLOM YANG TIDAK DIPAKAI
# -----------------------------------------------------------

drop_columns = [
    'No_Reff',
    'Timestamp',
    'Tanggal',
    'Nama_Penerima',
    'Nama_Pengirim',
    'Jenis_Transaksi',
    'Quarter_Label',
    'Warning_Level',
    'Kategori_Pembayaran',
    'Minggu_Ke',
]

df_ews = df.drop(columns=drop_columns)

print(f"✓ Kolom yang di-drop: {len(drop_columns)}")

# -----------------------------------------------------------
# 2. DEFINISIKAN FITUR (SEMUA YANG TERSISA)
# -----------------------------------------------------------

feature_columns = [
    # Temporal Features
    'Bulan',
    'Hari',
    'Hari_Minggu',
    'Quarter',
    'Is_Weekend',
    'Is_Akhir_Bulan',
    'Is_Awal_Bulan',
    'Hari_Dari_Awal_Bulan',

    # Behavior Features
    'Total_Transaksi',
    'Rata_Nominal',
    'Frekuensi_Per_Hari',
    'Durasi_Aktif_Hari',
    'Rata_Interval_Hari',
    'Jumlah_Terlambat',
    'Persentase_Terlambat',

    # Transaction Type
    'Is_TopUp',
    'Is_QRIS',
    'Is_Transfer',
    'Prop_TopUp',
    'Prop_QRIS',
    'Prop_Transfer',

    # Activity
    'Aktivitas_Bulan_Ini',
    'Aktivitas_Quarter_Ini',

    # Transaction Amount
    'Nominal_Transaksi',
]

target = 'Risk_Score'

print(f"\nJumlah fitur: {len(feature_columns)}")
print(f"Target: {target}")

# -----------------------------------------------------------
# 3. SPLIT DATA
# -----------------------------------------------------------

X = df_ews[feature_columns]
y = df_ews[target]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"\n✓ Train size: {len(X_train_reg):,}")
print(f"✓ Test size:  {len(X_test_reg):,}")

# -----------------------------------------------------------
# 4. SAMPLE DATA
# -----------------------------------------------------------

print(f"\n{'='*60}")
print("SAMPLE X_train (5 baris):")
print(X_train_reg.head())

print("\nSAMPLE y_train (5 baris):")
print(y_train_reg.head())

# -----------------------------------------------------------
# 5. STATISTIK
# -----------------------------------------------------------

print(f"\n{'='*60}")
print("STATISTIK FITUR")
print(f"{'='*60}")
print(X_train_reg.describe())

print(f"\n{'='*60}")
print("STATISTIK TARGET (Risk_Score)")
print(f"{'='*60}")
print(y_train_reg.describe())

print(f"{'='*60}")

PERSIAPAN DATA EWS (Minimal Features untuk Inferensi)
✓ Kolom yang di-drop: 10

Jumlah fitur: 24
Target: Risk_Score

✓ Train size: 7,200
✓ Test size:  1,800

SAMPLE X_train (5 baris):
      Bulan  Hari  Hari_Minggu  Quarter  Is_Weekend  Is_Akhir_Bulan  \
6317      1    17            4        1           0               0   
740       1    31            4        1           0               1   
3781      1    29            2        1           0               1   
7850      1    20            0        1           0               0   
2963      1    29            2        1           0               1   

      Is_Awal_Bulan  Hari_Dari_Awal_Bulan  Total_Transaksi  Rata_Nominal  ...  \
6317              0                    17               63  89782.126984  ...   
740               0                    31               76  69190.381579  ...   
3781              0                    29               67  82550.253731  ...   
7850              0                    20               56  82653

In [4]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [18]:
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import joblib
import os
import json
from datetime import datetime


# ===================================================================
# TRAINING BASE MODELS
# ===================================================================
print("\n" + "="*70)
print("TRAINING BASE MODELS (REGRESSION)")
print("="*70)


# -------------------------------------------------------------
# 1. XGBoost Regressor
# -------------------------------------------------------------
print("\n1. Training XGBoost...")

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(
    X_train_reg, y_train_reg,
    eval_set=[(X_test_reg, y_test_reg)],
    verbose=False
)

xgb_pred_train = xgb_model.predict(X_train_reg)
xgb_pred_test  = xgb_model.predict(X_test_reg)

xgb_mae = mean_absolute_error(y_test_reg, xgb_pred_test)
xgb_rmse = np.sqrt(mean_squared_error(y_test_reg, xgb_pred_test))
xgb_r2 = r2_score(y_test_reg, xgb_pred_test)

print(f"   MAE:  {xgb_mae:.4f}")
print(f"   RMSE: {xgb_rmse:.4f}")
print(f"   R2:   {xgb_r2:.4f}")


# -------------------------------------------------------------
# 2. CatBoost Regressor
# -------------------------------------------------------------
print("\n2. Training CatBoost...")

cat_model = CatBoostRegressor(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    loss_function='RMSE',
    random_state=42,
    verbose=False
)

cat_model.fit(
    X_train_reg, y_train_reg,
    eval_set=(X_test_reg, y_test_reg),
    early_stopping_rounds=30
)

cat_pred_train = cat_model.predict(X_train_reg)
cat_pred_test  = cat_model.predict(X_test_reg)

cat_mae = mean_absolute_error(y_test_reg, cat_pred_test)
cat_rmse = np.sqrt(mean_squared_error(y_test_reg, cat_pred_test))
cat_r2 = r2_score(y_test_reg, cat_pred_test)

print(f"   MAE:  {cat_mae:.4f}")
print(f"   RMSE: {cat_rmse:.4f}")
print(f"   R2:   {cat_r2:.4f}")


# -------------------------------------------------------------
# 3. LightGBM Regressor
# -------------------------------------------------------------
print("\n3. Training LightGBM...")

lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='regression',
    metric='rmse',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(
    X_train_reg, y_train_reg,
    eval_set=[(X_test_reg, y_test_reg)],
    callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
)

lgb_pred_train = lgb_model.predict(X_train_reg)
lgb_pred_test  = lgb_model.predict(X_test_reg)

lgb_mae = mean_absolute_error(y_test_reg, lgb_pred_test)
lgb_rmse = np.sqrt(mean_squared_error(y_test_reg, lgb_pred_test))
lgb_r2 = r2_score(y_test_reg, lgb_pred_test)

print(f"   MAE:  {lgb_mae:.4f}")
print(f"   RMSE: {lgb_rmse:.4f}")
print(f"   R2:   {lgb_r2:.4f}")


# ===================================================================
# META-LEARNER (STACKING REGRESSOR)
# ===================================================================
print("\n" + "="*70)
print("TRAINING META-LEARNER (STACKING REGRESSION)")
print("="*70)

# Stack predictions from all base models
meta_train = np.column_stack([xgb_pred_train, cat_pred_train, lgb_pred_train])
meta_test  = np.column_stack([xgb_pred_test,  cat_pred_test,  lgb_pred_test])

meta_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

meta_model.fit(meta_train, y_train_reg)

meta_pred = meta_model.predict(meta_test)

# ===================================================================
# FINAL EVALUATION
# ===================================================================
meta_mae = mean_absolute_error(y_test_reg, meta_pred)
meta_rmse = np.sqrt(mean_squared_error(y_test_reg, meta_pred))
meta_r2 = r2_score(y_test_reg, meta_pred)

print("\n" + "="*70)
print("FINAL EVALUATION (STACKING REGRESSION)")
print("="*70)
print(f"MAE:  {meta_mae:.4f}")
print(f"RMSE: {meta_rmse:.4f}")
print(f"R2:   {meta_r2:.4f}")


# ===================================================================
# COMPARISON TABLE
# ===================================================================
print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)

comparison = pd.DataFrame({
    'Model': ['XGBoost', 'CatBoost', 'LightGBM', 'Stacking (Meta)'],
    'MAE': [xgb_mae, cat_mae, lgb_mae, meta_mae],
    'RMSE': [xgb_rmse, cat_rmse, lgb_rmse, meta_rmse],
    'R2': [xgb_r2, cat_r2, lgb_r2, meta_r2]
})

print(comparison.to_string(index=False))


# ===================================================================
# FEATURE IMPORTANCE (XGBoost)
# ===================================================================
print("\n" + "="*70)
print("TOP 10 FEATURE IMPORTANCE (XGBoost)")
print("="*70)

fi = pd.DataFrame({
    'Feature': X_train_reg.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(fi.head(10).to_string(index=False))


# ===================================================================
# SAVE MODELS FOR INFERENCE
# ===================================================================
print("\n" + "="*70)
print("SAVING MODELS FOR INFERENCE")
print("="*70)

model_dir = "/content/drive/MyDrive/PBL SEM 5/Dataset ML/models_ews/Regression"
os.makedirs(model_dir, exist_ok=True)

# 1. Save Base Models
joblib.dump(xgb_model, f"{model_dir}/xgb_regressor.pkl")
joblib.dump(cat_model, f"{model_dir}/cat_regressor.pkl")
joblib.dump(lgb_model, f"{model_dir}/lgb_regressor.pkl")
print("✓ Base models saved")

# 2. Save Meta Model
joblib.dump(meta_model, f"{model_dir}/meta_regressor.pkl")
print("✓ Meta model saved")

# 3. Save Feature Importance
fi.to_csv(f"{model_dir}/feature_importance.csv", index=False)
print("✓ Feature importance saved")

# 4. Save Model Info & Metrics
model_info = {
    "model_version": "1.0",
    "training_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "type": "regression",
    "target": "Risk_Score",

    # Features
    "feature_columns": list(X_train_reg.columns),
    "num_features": len(X_train_reg.columns),

    # Models
    "base_models": ["xgb_regressor", "cat_regressor", "lgb_regressor"],
    "meta_model": "meta_regressor",

    # Metrics
    "metrics": {
        "xgboost": {"MAE": float(xgb_mae), "RMSE": float(xgb_rmse), "R2": float(xgb_r2)},
        "catboost": {"MAE": float(cat_mae), "RMSE": float(cat_rmse), "R2": float(cat_r2)},
        "lightgbm": {"MAE": float(lgb_mae), "RMSE": float(lgb_rmse), "R2": float(lgb_r2)},
        "stacking": {"MAE": float(meta_mae), "RMSE": float(meta_rmse), "R2": float(meta_r2)}
    },

    # Training data info
    "train_samples": len(X_train_reg),
    "test_samples": len(X_test_reg),

    # Feature statistics for inference (untuk handle missing values)
    "feature_stats": {
        col: {
            "mean": float(X_train_reg[col].mean()),
            "std": float(X_train_reg[col].std()),
            "min": float(X_train_reg[col].min()),
            "max": float(X_train_reg[col].max())
        }
        for col in X_train_reg.columns
    }
}

with open(f"{model_dir}/model_info.json", "w") as f:
    json.dump(model_info, f, indent=4)
print("✓ Model info saved")

# 5. Save Evaluation Results
evaluation = {
    "comparison": comparison.to_dict(orient='records'),
    "best_model": "Stacking (Meta)" if meta_mae == min([xgb_mae, cat_mae, lgb_mae, meta_mae]) else None,
    "feature_importance_top10": fi.head(10).to_dict(orient='records')
}

with open(f"{model_dir}/evaluation.json", "w") as f:
    json.dump(evaluation, f, indent=4)
print("✓ Evaluation results saved")

print("\n" + "="*70)
print("ALL ARTIFACTS SAVED TO:")
print(f"   {model_dir}")
print("="*70)
print("\nSaved files:")
print("   1. xgb_regressor.pkl")
print("   2. cat_regressor.pkl")
print("   3. lgb_regressor.pkl")
print("   4. meta_regressor.pkl")
print("   5. model_info.json (dengan feature stats)")
print("   6. feature_importance.csv")
print("   7. evaluation.json")
print("\n" + "="*70)


TRAINING BASE MODELS (REGRESSION)

1. Training XGBoost...
   MAE:  0.0291
   RMSE: 0.0608
   R2:   1.0000

2. Training CatBoost...
   MAE:  0.0345
   RMSE: 0.0627
   R2:   1.0000

3. Training LightGBM...
   MAE:  0.0540
   RMSE: 0.0861
   R2:   1.0000

TRAINING META-LEARNER (STACKING REGRESSION)

FINAL EVALUATION (STACKING REGRESSION)
MAE:  0.0150
RMSE: 0.0474
R2:   1.0000

MODEL COMPARISON
          Model      MAE     RMSE       R2
        XGBoost 0.029060 0.060770 0.999985
       CatBoost 0.034515 0.062718 0.999984
       LightGBM 0.054021 0.086076 0.999971
Stacking (Meta) 0.015010 0.047405 0.999991

TOP 10 FEATURE IMPORTANCE (XGBoost)
             Feature  Importance
       Is_Awal_Bulan    0.394569
Hari_Dari_Awal_Bulan    0.223651
                Hari    0.149649
      Is_Akhir_Bulan    0.101875
          Is_Weekend    0.073813
         Hari_Minggu    0.033931
               Bulan    0.012419
Persentase_Terlambat    0.006662
    Jumlah_Terlambat    0.001903
 Aktivitas_Bulan_Ini   

In [20]:
!pip install questionary

In [23]:
import json
import joblib
import numpy as np
import pandas as pd
from datetime import datetime

# ==========================================================
# 1. LOAD MODELS & MODEL INFO
# ==========================================================
print("\n" + "="*70)
print("  SISTEM PREDIKSI EARLY WARNING SYSTEM (EWS)")
print("  Prediksi Risiko Keterlambatan Pembayaran")
print("="*70)

print("\n⏳ Loading models...")
model_dir = "/content/drive/MyDrive/PBL SEM 5/Dataset ML/models_ews/Regression"

try:
    models = {
        "xgb": joblib.load(f"{model_dir}/xgb_regressor.pkl"),
        "cat": joblib.load(f"{model_dir}/cat_regressor.pkl"),
        "lgb": joblib.load(f"{model_dir}/lgb_regressor.pkl")
    }

    meta_model = joblib.load(f"{model_dir}/meta_regressor.pkl")

    with open(f"{model_dir}/model_info.json", "r") as f:
        model_info = json.load(f)

    feature_columns = model_info["feature_columns"]
    feature_stats = model_info["feature_stats"]

    print(f"✓ Models loaded successfully")
    print(f"✓ Using {len(feature_columns)} features")
    print(f"✓ Model version: {model_info['model_version']}")
    print(f"✓ Trained on: {model_info['training_date']}")

except Exception as e:
    print(f"\n❌ ERROR: Gagal memuat model!")
    print(f"Detail: {str(e)}")
    raise


# ==========================================================
# 2. BENTUK FITUR DARI INPUT ADMIN
# ==========================================================
def build_base_features(tanggal: str, nominal: int):
    """
    Membentuk fitur dari input admin.
    - Fitur temporal: dari tanggal input
    - Fitur nominal: dari input nominal
    - Fitur behavior: gunakan mean dari training (karena tidak ada history)
    """
    dt = datetime.strptime(tanggal, "%Y-%m-%d")

    features = {
        # === TEMPORAL FEATURES (dari input tanggal) ===
        "Bulan": dt.month,
        "Hari": dt.day,
        "Hari_Minggu": dt.weekday(),
        "Quarter": (dt.month - 1) // 3 + 1,
        "Is_Weekend": 1 if dt.weekday() >= 5 else 0,
        "Is_Akhir_Bulan": 1 if dt.day >= 28 else 0,
        "Is_Awal_Bulan": 1 if dt.day <= 3 else 0,
        "Hari_Dari_Awal_Bulan": dt.day,

        # === NOMINAL (dari input) ===
        "Nominal_Transaksi": nominal,

        # === BEHAVIOR FEATURES (gunakan mean dari training) ===
        "Total_Transaksi": feature_stats["Total_Transaksi"]["mean"],
        "Rata_Nominal": feature_stats["Rata_Nominal"]["mean"],
        "Frekuensi_Per_Hari": feature_stats["Frekuensi_Per_Hari"]["mean"],
        "Durasi_Aktif_Hari": feature_stats["Durasi_Aktif_Hari"]["mean"],
        "Rata_Interval_Hari": feature_stats["Rata_Interval_Hari"]["mean"],
        "Jumlah_Terlambat": feature_stats["Jumlah_Terlambat"]["mean"],
        "Persentase_Terlambat": feature_stats["Persentase_Terlambat"]["mean"],

        # === TRANSACTION TYPE (gunakan mean dari training) ===
        "Is_TopUp": feature_stats["Is_TopUp"]["mean"],
        "Is_QRIS": feature_stats["Is_QRIS"]["mean"],
        "Is_Transfer": feature_stats["Is_Transfer"]["mean"],

        # === PROPORSI TRANSAKSI (gunakan mean dari training) ===
        "Prop_TopUp": feature_stats["Prop_TopUp"]["mean"],
        "Prop_QRIS": feature_stats["Prop_QRIS"]["mean"],
        "Prop_Transfer": feature_stats["Prop_Transfer"]["mean"],

        # === AKTIVITAS (gunakan mean dari training) ===
        "Aktivitas_Bulan_Ini": feature_stats["Aktivitas_Bulan_Ini"]["mean"],
        "Aktivitas_Quarter_Ini": feature_stats["Aktivitas_Quarter_Ini"]["mean"],
    }

    return features


# ==========================================================
# 3. PERSIAPAN FITUR SESUAI TRAINING
# ==========================================================
def prepare_features(input_dict):
    """
    Pastikan urutan fitur sama dengan training.
    """
    df = pd.DataFrame([[input_dict.get(col, 0) for col in feature_columns]],
                      columns=feature_columns)
    return df.values


# ==========================================================
# 4. STACKING PREDICTION
# ==========================================================
def predict_ews(tanggal: str, nominal: int):
    """
    Prediksi risiko terlambat menggunakan stacking ensemble.
    """
    base_dict = build_base_features(tanggal, nominal)
    X = prepare_features(base_dict)

    # Base predictions
    preds_base = np.column_stack([
        models["xgb"].predict(X),
        models["cat"].predict(X),
        models["lgb"].predict(X)
    ])

    # Meta model → Final prediction
    final_pred = meta_model.predict(preds_base)[0]

    return float(final_pred)


# ==========================================================
# 5. TAMPILKAN HASIL
# ==========================================================
def tampilkan_hasil(risiko, tanggal, nominal, target_type, rt_number=None):
    """
    Menampilkan hasil prediksi dengan peringatan
    """
    print("\n" + "="*70)
    print("                  HASIL PREDIKSI RISIKO TERLAMBAT")
    print("="*70)

    # Info input
    print(f"\n📅 Tanggal      : {tanggal}")
    print(f"💰 Nominal      : Rp {nominal:,}")

    if target_type == "broadcast":
        print(f"📡 Target       : BROADCAST (Semua RT)")
    else:
        print(f"🏠 Target       : RT {rt_number}")

    print("\n" + "-"*70)

    # Kategori risiko
    if risiko < 20:
        status = "RENDAH"
        emoji = "✅"
        color_code = "\033[92m"  # Hijau
        rekomendasi = "Risiko rendah. Transaksi dapat dilanjutkan dengan aman."
        tindakan = "• Lakukan monitoring rutin"
    elif risiko < 50:
        status = "SEDANG"
        emoji = "⚠️"
        color_code = "\033[93m"  # Kuning
        rekomendasi = "Risiko sedang. Perlu monitoring berkala."
        tindakan = "• Monitor pembayaran secara berkala\n• Kirim reminder H-3 jatuh tempo"
    elif risiko < 75:
        status = "TINGGI"
        emoji = "🔴"
        color_code = "\033[91m"  # Merah
        rekomendasi = "PERINGATAN: Risiko tinggi keterlambatan!"
        tindakan = "• Aktifkan reminder otomatis\n• Follow-up intensif H-7 dan H-3\n• Pertimbangkan metode pembayaran alternatif"
    else:
        status = "SANGAT TINGGI"
        emoji = "🚨"
        color_code = "\033[91m\033[1m"  # Merah bold
        rekomendasi = "PERINGATAN KRITIS! Risiko sangat tinggi!"
        tindakan = "• TUNDA transaksi jika memungkinkan\n• Follow-up personal sebelum transaksi\n• Siapkan prosedur penagihan\n• Pertimbangkan pembayaran di muka"

    print(f"{color_code}🎯 Risiko       : {risiko:.2f}%\033[0m")
    print(f"{color_code}📊 Status       : {emoji} {status}\033[0m")
    print(f"\n💡 Rekomendasi  : {rekomendasi}")
    print(f"\n📋 Tindakan yang disarankan:")
    print(f"{tindakan}")

    print("\n" + "="*70)


# ==========================================================
# 6. MENU UTAMA
# ==========================================================
def main_menu():
    """
    Menu utama untuk input prediksi EWS
    """
    print("\n")

    try:
        # ===== STEP 1: Pilih Target =====
        print("="*70)
        print("STEP 1: PILIH TARGET PENGIRIMAN")
        print("="*70)
        print("\n  1. Broadcast (Semua RT)")
        print("  2. RT Tertentu")

        while True:
            pilihan = input("\n📡 Pilih opsi (1/2): ").strip()
            if pilihan == "1":
                target_type = "broadcast"
                rt_number = None
                print("✓ Target: BROADCAST")
                break
            elif pilihan == "2":
                target_type = "rt_tertentu"
                while True:
                    rt_number = input("🏠 Masukkan nomor RT: ").strip()
                    if rt_number:
                        print(f"✓ Target: RT {rt_number}")
                        break
                    else:
                        print("❌ Nomor RT tidak boleh kosong!")
                break
            else:
                print("❌ Pilihan tidak valid! Masukkan 1 atau 2.")

        # ===== STEP 2: Input Tanggal =====
        print("\n" + "="*70)
        print("STEP 2: MASUKKAN TANGGAL TRANSAKSI")
        print("="*70)

        default_tanggal = datetime.now().strftime("%Y-%m-%d")
        print(f"\nFormat: YYYY-MM-DD (contoh: {default_tanggal})")

        while True:
            tanggal = input(f"📅 Tanggal [{default_tanggal}]: ").strip()
            if not tanggal:
                tanggal = default_tanggal

            try:
                datetime.strptime(tanggal, "%Y-%m-%d")
                print(f"✓ Tanggal: {tanggal}")
                break
            except ValueError:
                print("❌ Format tanggal salah! Gunakan format YYYY-MM-DD")

        # ===== STEP 3: Input Nominal =====
        print("\n" + "="*70)
        print("STEP 3: MASUKKAN NOMINAL TRANSAKSI")
        print("="*70)

        while True:
            nominal_input = input("\n💰 Nominal (Rp): ").strip().replace(".", "").replace(",", "")

            if not nominal_input:
                print("❌ Nominal tidak boleh kosong!")
                continue

            if not nominal_input.isdigit():
                print("❌ Masukkan angka yang valid!")
                continue

            nominal = int(nominal_input)
            if nominal <= 0:
                print("❌ Nominal harus lebih dari 0!")
                continue

            print(f"✓ Nominal: Rp {nominal:,}")
            break

        # ===== STEP 4: Prediksi =====
        print("\n" + "="*70)
        print("STEP 4: MENGHITUNG PREDIKSI")
        print("="*70)
        print("\n⏳ Memproses prediksi...")

        try:
            risiko = predict_ews(tanggal, nominal)
            tampilkan_hasil(risiko, tanggal, nominal, target_type, rt_number)
        except Exception as e:
            print(f"\n❌ ERROR: Gagal melakukan prediksi!")
            print(f"Detail: {str(e)}")
            return

        # ===== Opsi Ulang =====
        print("\n" + "="*70)
        ulang = input("🔄 Prediksi transaksi lain? (y/n): ").strip().lower()

        if ulang == 'y':
            main_menu()
        else:
            print("\n" + "="*70)
            print("  👋 Terima kasih telah menggunakan sistem EWS!")
            print("="*70 + "\n")

    except KeyboardInterrupt:
        print("\n\n❌ Program dihentikan oleh user.\n")
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}\n")


# ==========================================================
# 7. JALANKAN PROGRAM
# ==========================================================
if __name__ == "__main__":
    try:
        main_menu()
    except KeyboardInterrupt:
        print("\n\n👋 Program dihentikan.\n")


  SISTEM PREDIKSI EARLY WARNING SYSTEM (EWS)
  Prediksi Risiko Keterlambatan Pembayaran

⏳ Loading models...
✓ Models loaded successfully
✓ Using 24 features
✓ Model version: 1.0
✓ Trained on: 2025-11-16 10:51:29


STEP 1: PILIH TARGET PENGIRIMAN

  1. Broadcast (Semua RT)
  2. RT Tertentu

📡 Pilih opsi (1/2): 1
✓ Target: BROADCAST

STEP 2: MASUKKAN TANGGAL TRANSAKSI

Format: YYYY-MM-DD (contoh: 2025-11-16)
📅 Tanggal [2025-11-16]: 2025-11-25
✓ Tanggal: 2025-11-25

STEP 3: MASUKKAN NOMINAL TRANSAKSI

💰 Nominal (Rp): 12000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✓ Nominal: Rp 12,000

STEP 4: MENGHITUNG PREDIKSI

⏳ Memproses prediksi...

                  HASIL PREDIKSI RISIKO TERLAMBAT

📅 Tanggal      : 2025-11-25
💰 Nominal      : Rp 12,000
📡 Target       : BROADCAST (Semua RT)

----------------------------------------------------------------------
🎯 Risiko       : 63.81%
📊 Status       : 🔴 TINGGI

💡 Rekomendasi  : PERINGATAN: Risiko tinggi keterlambatan!

📋 Tindakan yang disarankan:
• Aktifkan reminder otomatis
• Follow-up intensif H-7 dan H-3
• Pertimbangkan metode pembayaran alternatif


🔄 Prediksi transaksi lain? (y/n): y


STEP 1: PILIH TARGET PENGIRIMAN

  1. Broadcast (Semua RT)
  2. RT Tertentu

📡 Pilih opsi (1/2): 1
✓ Target: BROADCAST

STEP 2: MASUKKAN TANGGAL TRANSAKSI

Format: YYYY-MM-DD (contoh: 2025-11-16)
📅 Tanggal [2025-11-16]: 2025-12-02
✓ Tanggal: 2025-12-02

STEP 3: MASUKKAN NOMINAL TRANSAKSI

💰 Nominal (Rp): 12000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✓ Nominal: Rp 12,000

STEP 4: MENGHITUNG PREDIKSI

⏳ Memproses prediksi...

                  HASIL PREDIKSI RISIKO TERLAMBAT

📅 Tanggal      : 2025-12-02
💰 Nominal      : Rp 12,000
📡 Target       : BROADCAST (Semua RT)

----------------------------------------------------------------------
🎯 Risiko       : 28.50%
📊 Status       : ⚠️ SEDANG

💡 Rekomendasi  : Risiko sedang. Perlu monitoring berkala.

📋 Tindakan yang disarankan:
• Monitor pembayaran secara berkala
• Kirim reminder H-3 jatuh tempo


🔄 Prediksi transaksi lain? (y/n): y


STEP 1: PILIH TARGET PENGIRIMAN

  1. Broadcast (Semua RT)
  2. RT Tertentu

📡 Pilih opsi (1/2): 1
✓ Target: BROADCAST

STEP 2: MASUKKAN TANGGAL TRANSAKSI

Format: YYYY-MM-DD (contoh: 2025-11-16)
📅 Tanggal [2025-11-16]: 2026-01-02
✓ Tanggal: 2026-01-02

STEP 3: MASUKKAN NOMINAL TRANSAKSI

💰 Nominal (Rp): 12000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✓ Nominal: Rp 12,000

STEP 4: MENGHITUNG PREDIKSI

⏳ Memproses prediksi...

                  HASIL PREDIKSI RISIKO TERLAMBAT

📅 Tanggal      : 2026-01-02
💰 Nominal      : Rp 12,000
📡 Target       : BROADCAST (Semua RT)

----------------------------------------------------------------------
🎯 Risiko       : 28.50%
📊 Status       : ⚠️ SEDANG

💡 Rekomendasi  : Risiko sedang. Perlu monitoring berkala.

📋 Tindakan yang disarankan:
• Monitor pembayaran secara berkala
• Kirim reminder H-3 jatuh tempo


🔄 Prediksi transaksi lain? (y/n): y


STEP 1: PILIH TARGET PENGIRIMAN

  1. Broadcast (Semua RT)
  2. RT Tertentu

📡 Pilih opsi (1/2): 1
✓ Target: BROADCAST

STEP 2: MASUKKAN TANGGAL TRANSAKSI

Format: YYYY-MM-DD (contoh: 2025-11-16)
📅 Tanggal [2025-11-16]: 2026-03-05
✓ Tanggal: 2026-03-05

STEP 3: MASUKKAN NOMINAL TRANSAKSI

💰 Nominal (Rp): 12000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✓ Nominal: Rp 12,000

STEP 4: MENGHITUNG PREDIKSI

⏳ Memproses prediksi...

                  HASIL PREDIKSI RISIKO TERLAMBAT

📅 Tanggal      : 2026-03-05
💰 Nominal      : Rp 12,000
📡 Target       : BROADCAST (Semua RT)

----------------------------------------------------------------------
🎯 Risiko       : 29.86%
📊 Status       : ⚠️ SEDANG

💡 Rekomendasi  : Risiko sedang. Perlu monitoring berkala.

📋 Tindakan yang disarankan:
• Monitor pembayaran secara berkala
• Kirim reminder H-3 jatuh tempo


🔄 Prediksi transaksi lain? (y/n): n

  👋 Terima kasih telah menggunakan sistem EWS!

